### to_thread

return coro that runs func in separate thread.
- args, kwargs can be passed (B)
- could run blocking IO bound funcs (C), or CPU bound funcs that release GIL (D)

In [ ]:
import time
import asyncio


def blocking_io(sec):
    time.sleep(sec)  # C


async def main():
    t = time.time()
    await asyncio.gather(
        asyncio.to_thread(blocking_io, 1),  # B
        asyncio.to_thread(blocking_io, sec=1),
    )
    print(f"{time.time() - t = }")


await main()

time.time() - t = 1.0009491443634033


In [ ]:
import numpy as np
from scipy import optimize


def optimization_task(n):  # D 1
    def objective(x):
        return np.sum(x**2) + np.sum(np.sin(x))

    x0 = np.random.randn(n)
    result = optimize.minimize(objective, x0, method="BFGS")
    return result.x


t = time.time()
_ = optimization_task(1000)
print(f"Blocking optimization took {time.time() - t} seconds.")

Blocking optimization took 1.0353281497955322 seconds.


In [32]:
async def main():
    t = time.time()
    await asyncio.gather(
        asyncio.to_thread(optimization_task, 1000),
        asyncio.to_thread(optimization_task, n=1000),
    )
    print(f"{time.time() - t = }")


await main()

time.time() - t = 2.0410995483398438


In [ ]:
import numpy as np
import numba


@numba.jit(nopython=True, nogil=True)  # D 2
def gradient_descent(x0, lr=0.01, iterations=10000):
    """Simple gradient descent optimization with Numba"""
    x = x0.copy()
    for _ in range(iterations):
        grad = np.zeros_like(x)
        for i in range(len(x)):
            grad[i] = 2 * x[i] + np.cos(x[i])
        x = x - lr * grad
    return x


def optimization_task_numba(n):  # D
    x0 = np.random.randn(n)
    result = gradient_descent(x0)
    return result


# Test
t = time.time()
_ = optimization_task_numba(1_000 * 30)
print(f"Blocking optimization took {time.time() - t} seconds.")

Blocking optimization took 1.0260639190673828 seconds.


In [51]:
async def main():
    t = time.time()
    await asyncio.gather(
        asyncio.to_thread(optimization_task_numba, 1000 * 30),
        asyncio.to_thread(optimization_task_numba, 1000 * 30),
    )
    print(f"{time.time() - t = }")


await main()

time.time() - t = 0.9543752670288086
